In [1]:
from articlecommon import *
import warnings
warnings.filterwarnings("ignore")
from category_encoders import TargetEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import SequentialFeatureSelector

# Add Policy_Sales_Channel OHE columns
psc_ohe = pd.get_dummies(X_le["Policy_Sales_Channel"], prefix="Policy_Sales_Channel")
counts = X_le["Policy_Sales_Channel"].value_counts()
counts = counts[counts == 1]
columns_to_remove = list(map(lambda x: "Policy_Sales_Channel_" + str(x), counts.index.tolist()))
psc_ohe = psc_ohe.drop(columns=columns_to_remove)

X_le_psc = pd.concat([X_le, psc_ohe], axis=1)

value_counts = X_le_psc["Policy_Sales_Channel"].value_counts().index
columns_to_remove = list(map(lambda x: "Policy_Sales_Channel_" + str(value_counts[x]), [*range(147, 86, -1)]))
columns_to_remove += ['Vintage']

X_final = X_le_psc.drop(columns=columns_to_remove)

X_final['Policy_Sales_Channel_te'] = X_final['Policy_Sales_Channel']

encoder = TargetEncoder(cols=["Policy_Sales_Channel", "Region_Code"])
quant_trans_uniform = QuantileTransformer(output_distribution='uniform')
lgbm = LGBMClassifier(random_state=42, verbose=-1)

model = make_pipeline(encoder, quant_trans_uniform, lgbm)

In [2]:
# TODO: implement multithreading

def select_features(model, X, y, fixed_columns, path):
    selection_columns = [col for col in X.columns.values if col not in fixed_columns]
    scores = dict()
    counter = 0
    for x in selection_columns:
        X_to_count = X.drop(columns=x)
        score = cross_val_score(model, X_to_count, y, cv=10, scoring='roc_auc', n_jobs=-1).mean() #TODO: set cv=10 and the whole dataset
        scores[x] = score
        counter += 1
        print(str(counter) + ",", end='')
    column_to_remove = max(scores, key=scores.get)
    print()
    print("Obtained " + str(scores[column_to_remove]) + " without column " + column_to_remove)
    
    selected_columns = X.columns.values.tolist()
    selected_columns.remove(column_to_remove)
    selected_cols_s = pd.Series(selected_columns)
    selected_cols_s.to_csv(path)

In [3]:
fixed_columns = ['Gender', 'Age', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium', 'Policy_Sales_Channel', 'Policy_Sales_Channel_te']
# select_features(model, X_final, y, fixed_columns, 'results/own96.csv')
# for x in range(95, 18, -1):
#     select_features(model, X_final[pd.read_csv("results/own" + str(x+1) + ".csv")["0"]], y, fixed_columns, 'results/own' + str(x) + '.csv')

1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,
Obtained 0.8582028195895045 without column Policy_Sales_Channel_52.0
1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,
Obtained 0.8582257649628018 without column Policy_Sales_Channel_16.0
1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,
Obtained 0.8582586959303461 without column Policy_Sales_Channel_22.0
1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,1

KeyboardInterrupt: 

In [ ]:
for x in range(85, 10, -1):
    select_features(model, X_final[pd.read_csv("results/own" + str(x+1) + ".csv")["0"]], y, fixed_columns, 'results/own' + str(x) + '.csv')